<!-- <div align='center' ><font color = blue size='8' face = '黑体'>黄河口悬浮泥沙分布在线反演APP</font></div>
<div align='right' ><font color = green size='2' face = '黑体'>------by xiaoming</font></div>
<div align='left' ><font size='5' face = '楷体'> 
    1.通过云量筛选指定日期黄河口的影像<br>
    2.利用内置的悬沙反演算法将选择的影像执行反演程序，显示反演结果<br>
    3.将反演结果进行简单的制图（指北针、图例、比例尺）<br><br><br>
    界面加载速度受网络影响，请等候.......
    </font></div>                                                                       -->

<div align='center' ><font color = #66CCCC size='7' face = '楷体' ><b>黄河口悬浮泥沙分布在线反演APP</b></font></div>

**可实现:**
<font size='5' face = '楷体'> 
1. 通过云量筛选指定日期黄河口的影像
2. 利用内置的悬沙反演算法将选择的影像执行反演程序，显示反演结果 
3. 将反演结果进行简单的制图（指北针、图例、比例尺）【仅限于本地版本】
4. 制作遥感影像和反演结果的gif动图
5. 利用已知实测数据的点文件对反演结果进行精度验证


<br><br>
**Contact： Xiaoming**  (https://suxiaoming.tech/) 


<br>
界面加载速度受网络影响，请稍等......

In [1]:
import os
os.environ["HTTP_PROXY"]="http://127.0.0.1:4780"
os.environ["HTTPS_PROXY"]="http://127.0.0.1:4780"

import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
from ipywidgets import *
# import matplotlib.pyplot as plt
# # from geemap import cartoee
# from matplotlib.lines import Line2D
import matplotlib
matplotlib.use('Agg')
import tkinter as tk
from tkinter import filedialog
import math
import numpy as np
import pandas as pd
import win32api,win32con

In [3]:
Map = geemap.Map(center=[37.7,118.81],zoom=9)
Map

Map(center=[37.7, 118.81], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(childre…

In [4]:
# 添加卫星影像数据，卫星影像按照云量进行筛选的函数
def cloud_select(year,cloud):
    if 1984<= year <=2011:
        start_y = ee.Date.fromYMD(year,1,1)
        end_y = start_y.advance(1,'year')
        images = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')\
              .filterDate(start_y,end_y)\
              .filter(ee.Filter.eq('WRS_ROW',34))\
              .filter(ee.Filter.eq('WRS_PATH',121))\
              .filter(ee.Filter.lte('CLOUD_COVER',cloud))

        sys_id = images.aggregate_array('system:id').getInfo()
        cloud = images.aggregate_array('CLOUD_COVER').getInfo()
        # 显示影响的序列号加上云量A
        result = []
        for i in range(len(sys_id)):
            result.append(sys_id[i] + ' : '+ str(cloud[i]))
        return result,sys_id
    elif 2012 <= year <= 2014:
        
        start_y = ee.Date.fromYMD(year,1,1)
        end_y = start_y.advance(1,'year')
        images = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
              .filterDate(start_y,end_y)\
              .filter(ee.Filter.eq('WRS_ROW',34))\
              .filter(ee.Filter.eq('WRS_PATH',121))\
              .filter(ee.Filter.lte('CLOUD_COVER',cloud))

        sys_id = images.aggregate_array('system:id').getInfo()
        cloud = images.aggregate_array('CLOUD_COVER').getInfo()
        # 显示影响的序列号加上云量
        result = []
        for i in range(len(sys_id)):
            result.append(sys_id[i] + ' : '+ str(cloud[i]))
        return result,sys_id
    else:
        start_y = ee.Date.fromYMD(year,1,1)
        end_y = start_y.advance(1,'year')
        images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
              .filterDate(start_y,end_y)\
              .filter(ee.Filter.eq('WRS_ROW',34))\
              .filter(ee.Filter.eq('WRS_PATH',121))\
              .filter(ee.Filter.lte('CLOUD_COVER',cloud))

        sys_id = images.aggregate_array('system:id').getInfo()
        cloud = images.aggregate_array('CLOUD_COVER').getInfo()
        # 显示影响的序列号加上云量
        result = []
        for i in range(len(sys_id)):
            result.append(sys_id[i] + ' : '+ str(cloud[i]))
        return result,sys_id

In [5]:
# Landsat 7去条带
def func_lvr(image):
  filled1a = image.focal_mean(2, 'square', 'pixels', 1)
  return filled1a.blend(image) 

# 反演SSC
def l5_ssc(image):
    sssc = image.expression(
        '1603.194 * (x*x*x) - 2677.011 * (x*x) + 2569.904 * x -368.906',{
            'x': image.select('B4').divide(image.select('B1'))
        }
    )
    return sssc

def l7_ssc(image):
    sssc = image.expression(
        '1502.239 * (x*x*x) - 2529.413 * (x*x) + 2490.969 * x -358.319',{
            'x': image.select('B4').divide(image.select('B1'))
        }
    )
    return sssc

def l8_ssc(image):
    sssc = image.expression(
        '1406.405 * (x*x*x) - 1824.106 * (x*x) + 2323.982 * x -268.034',{
            'x': image.select('B5').divide(image.select('B2'))
        }
    )
    return sssc

In [6]:
# 添加卫星影像数据，卫星影像按照云量进行云量最优筛选筛选的函数
def cloud_select_first(year):
    if 1984<= year <=2011:
        start_y = ee.Date.fromYMD(year,1,1)
        end_y = start_y.advance(1,'year')
        image = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')\
              .filterDate(start_y,end_y)\
              .filter(ee.Filter.eq('WRS_ROW',34))\
              .filter(ee.Filter.eq('WRS_PATH',121))\
              .sort('CLOUD_COVER').first()

        sys_id = image.get('system:id').getInfo()
        
        return sys_id
    elif 2012 <= year <= 2014:
        
        start_y = ee.Date.fromYMD(year,1,1)
        end_y = start_y.advance(1,'year')
        image = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
              .filterDate(start_y,end_y)\
              .filter(ee.Filter.eq('WRS_ROW',34))\
              .filter(ee.Filter.eq('WRS_PATH',121))\
              .sort('CLOUD_COVER').first()
                      
        sys_id = image.get('system:id').getInfo()
        return sys_id
    else:
        start_y = ee.Date.fromYMD(year,1,1)
        end_y = start_y.advance(1,'year')
        image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
              .filterDate(start_y,end_y)\
              .filter(ee.Filter.eq('WRS_ROW',34))\
              .filter(ee.Filter.eq('WRS_PATH',121))\
              .sort('CLOUD_COVER').first()
                      
        sys_id = image.get('system:id').getInfo()
        return sys_id
                      

In [7]:
def ui():
    global out,sscgif_btn,sscgif_btn,imagegif_btn,seletpath_btn,path,speed,\
    title,color,time_range,zero_btn,plot_btn,ssc_btn,image_btn,image_cb,label0,\
    result_ta,static_btn,cloud_slider,time_slider,image_select,output_widget,\
    sc_btn,shp_path,yanzheng_result,yanzheng_btn,exportimg_btn,exportssc_btn
    # 界面设计搭建
    style = {'description_width':'initial'}

    # 在地图右下角增加一个框用于显示文字或者图表
    output_widget = widgets.Output(layout={'border':'1px solid black'})

    # 小部件
    image_select = widgets.Dropdown(
        description = '选择数据源：',
        options=['Landsat','MODIS'],
        value = 'Landsat',
        style = style
    )

    time_slider = widgets.IntSlider(
        value = 1984,
        min = 1984,
        max = 2020,
        step = 1,
        description = '年份',
        orientation = 'horizontal',
        style = style
    )

    cloud_slider = widgets.FloatSlider(
        value = 10,
        min = 0,
        max = 20,
        step = 0.1,
        description = '云量',
        orientation = 'horizontal',
        style = style,
        readout_format = '.1f'
    )

    static_btn = widgets.Button(
        description = '按云量筛选',
        button_style = 'primary',
        tooltip = '单击',
        style = style
    )
    
    shp_path = widgets.Text(
        value='',
        placeholder='',
        description='',
        disabled=False   
    )

    label1 = widgets.Label(
        value='反演结果验证：'
    )
    
    label2 = widgets.Label(
        value='精度验证结果：'
    )
    
    shp_path = Text(description='', layout=Layout(width='450px'),style = style)
    yanzheng_result = Textarea(description='', layout=Layout(width='300px', height='120px'),style = style)
    result_ta = Textarea(description='', layout=Layout(width='600px', height='150px'),style = style)
    label0 = HTML(value="<b>结果（eg. id : cloud_cover）<b>")
    image_cb = Combobox(description='选择影像', layout=Layout(width='600px', height='auto'),style = style)
    image_btn = Button(description='显示影像', layout=Layout(width='auto',height='auto') ,button_style='info')
    ssc_btn = Button(description='反演SSC', layout=Layout(width='auto',height='auto') ,style=ButtonStyle(button_color='#FED154'))
    plot_btn = Button(description='制图', layout=Layout(width='auto',height='auto'), style=ButtonStyle(button_color='#88EA79'))
    zero_btn = Button(description='软件初始化', layout=Layout(width='auto',height='auto'), style=ButtonStyle(button_color='#E69494'))
    
    sc_btn = Button(description='导入实测点数据（.shp）', layout=Layout(width='auto',height='auto') ,style=ButtonStyle(button_color='#69A6FF'))
    yanzheng_btn = Button(description='精度验证', layout=Layout(width='auto',height='auto') ,style=ButtonStyle(button_color='#69A6FF'))
    exportimg_btn = Button(description='导出原始影像', layout=Layout(width='auto',height='auto') ,button_style='warning')
    exportssc_btn = Button(description='导出反演图像', layout=Layout(width='auto',height='auto') ,button_style='warning')

    # 3
    time_range = widgets.IntRangeSlider(
        value=[1984, 2021],
        min=1984,
        max=2021,
        step=1,
        description='选择年份范围',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True
    )

    color = widgets.ColorPicker(
        concise=False,
        description='字体颜色：',
        value='white',
        disabled=False
    )

    title = widgets.Text(
        value='Hello World',
        placeholder='Type something',
        description='标题(en)：',
        disabled=False   
    )

    speed = widgets.BoundedIntText(
        value=1,
        min=1,
        max=10,
        step=1,
        description='播放速度：',
        disabled=False
    )

    path = widgets.Text(
        value='',
        placeholder='',
        description='',
        disabled=False   
    )

    seletpath_btn = Button(description='选择保存位置', layout=Layout(width='auto',height='auto'), style=ButtonStyle(button_color='#E69494'))
    imagegif_btn = Button(description='影像GIF', layout=Layout(width='auto',height='auto'), style=ButtonStyle(button_color='#E69494'))
    sscgif_btn = Button(description='SSC-GIF', layout=Layout(width='auto',height='auto'), style=ButtonStyle(button_color='#E69494'))

    # 布局
    # 影响筛选
    f1 = widgets.HBox([
        widgets.VBox([image_select,time_slider,cloud_slider,static_btn]),
        widgets.VBox([
            widgets.HBox([label0]),
            widgets.HBox([result_ta])
            ],layout=Layout(
        border='2px solid green', padding='10px',
        align_items='stretch', width='auto'))
        ], layout=Layout(
        border='0px solid gray', padding='0px',
        align_items='stretch', width='auto'))

    # 影像反演
    f2 = widgets.VBox([
        widgets.HBox([image_cb, image_btn,ssc_btn,plot_btn]),
        widgets.HBox([label1, sc_btn, shp_path, yanzheng_btn]),
         widgets.HBox([label2, yanzheng_result]),
        widgets.HBox([exportimg_btn, exportssc_btn, zero_btn])
    ], layout=Layout(
        border='0px solid gray', padding='0px',
        align_items='stretch', width='auto'))

    # 变化动态图示
    f3 = widgets.VBox([
        widgets.HBox([time_range]),
        widgets.HBox([color,title,speed]),
        widgets.HBox([path,seletpath_btn]),
        widgets.HBox([imagegif_btn,sscgif_btn])
    ], layout=Layout(
        border='0px solid gray', padding='0px',
        align_items='stretch', width='auto'))

    # 将输出放在out中，便于清除
    out = widgets.Output(layout={'border': '0px solid black'})
    # with out:
    #     display(f1,f2)
    # out
    t = Tab(children=[f1, f2, f3], _titles={0: '影像筛选', 1: '影像反演', 2: '变化动态图示'})
    return t

In [8]:
bj = ui()
bj

In [9]:
# 点击事件
# 云量筛选
def submit_clicked(a):
    with output_widget:
        output_widget.clear_output()
        print('请稍等，正在查找中.......')
        Map.default_style = {'cursor': 'wait'}
    
        try:
            image = image_select.value
            year = time_slider.value
            cloud = cloud_slider.value
            output_widget.clear_output()
            if image == 'Landsat':
                result,sys_id = cloud_select(year,cloud)
                result_ta.value = str(result)
                image_cb.options = sys_id
        except Exception as e:
            print(e)
            print('查找过程出现错误，请重新输入参数！')
        Map.default_style = {'cursor': 'default'}


# 显示影像
def show_image(b):
    global image
    with output_widget:
        output_widget.clear_output()
        try:
            id = image_cb.value
            image = ee.Image(id)
#             判断是不是L7影像，进行去条带操作
            if id[11] == '7':
                image = func_lvr(image)
            
            vis_params = {
                'bands':['B4','B3','B2'],
                'min': 0,
                'max': 3000
            }
            Map.addLayer(image,vis_params,'select image')
            
        except Exception as e:
            print(e)
            print('加载失败')
            
# 显示SSC
def show_ssc(c):
    global ssc_intial
    with output_widget:
        output_widget.clear_output()
        try:
            print('123')
            id = image_cb.value
            image = ee.Image(id)
            #判断是不是L7影像，进行去条带操作
            if id[11] == '7':
                image = func_lvr(image)
            
            # 岸线变迁考虑在内
            land_path = 'users/bluesky666/yrd_ssc_coastline/' + str(time_slider.value) + 'land'
            ocean_path = 'users/bluesky666/yrd_ssc_coastline/' + str(time_slider.value) + 'sea'
            
            landshp = ee.FeatureCollection(land_path)
            oceanshp = ee.FeatureCollection(ocean_path)
            # 利用陆地和海洋shp分别裁剪出陆地和海洋范围
            land = image.clip(landshp)
            ocean = image.clip(oceanshp)

            # Set visualization parameters.
            vis_params0 = {'bands': ['B1'], 'min': 0.0, 'max': 100.0, 'opacity': 1.0, 'gamma': 1.0}
            vis_params = {'bands': ['constant'], 'palette': ['ff0000', ' ff0600', ' ff0c00', ' ff1200', ' ff1800', ' ff1e00', ' ff2300', ' ff2900', ' ff2f00', ' ff3500', ' ff3b00', ' ff4100', ' ff4700', ' ff4d00', ' ff5300', ' ff5900', ' ff5f00', ' ff6400', ' ff6a00', ' ff7000', ' ff7600', ' ff7c00', ' ff8200', ' ff8800', ' ff8e00', ' ff9400', ' ff9a00', ' ff9f00', ' ffa500', ' ffab00', ' ffb100', ' ffb700', ' ffbd00', ' ffc300', ' ffc900', ' ffcf00', ' ffd500', ' ffdb00', ' ffe000', ' ffe600', ' ffec00', ' fef100', ' fcf500', ' faf900', ' f8fd00', ' f4ff00', ' eeff00', ' e8ff00', ' e2ff00', ' ddff00', ' d7ff00', ' d1ff00', ' cbff00', ' c5ff00', ' bfff00', ' b9ff00', ' b3ff00', ' adff00', ' a7ff00', ' a2ff00', ' 9cff00', ' 96ff00', ' 90ff00', ' 8aff00', ' 84ff00', ' 7eff00', ' 78ff00', ' 72ff00', ' 6cff00', ' 66ff00', ' 61ff00', ' 5bff00', ' 55ff00', ' 4fff00', ' 49ff00', ' 43ff00', ' 3dff00', ' 37ff00', ' 31ff00', ' 2bff00', ' 25ff00', ' 20ff00', ' 1aff00', ' 14ff00', ' 0eff00', ' 08ff00', ' 06ff04', ' 04ff08', ' 02ff0c', ' 00ff10', ' 00ff16', ' 00ff1b', ' 00ff21', ' 00ff27', ' 00ff2d', ' 00ff33', ' 00ff39', ' 00ff3f', ' 00ff45', ' 00ff4b', ' 00ff51', ' 00ff57', ' 00ff5c', ' 00ff62', ' 00ff68', ' 00ff6e', ' 00ff74', ' 00ff7a', ' 00ff80', ' 00ff86', ' 00ff8c', ' 00ff92', ' 00ff97', ' 00ff9d', ' 00ffa3', ' 00ffa9', ' 00ffaf', ' 00ffb5', ' 00ffbb', ' 00ffc1', ' 00ffc7', ' 00ffcd', ' 00ffd3', ' 00ffd8', ' 00ffde', ' 00ffe4', ' 00ffea', ' 00fff0', ' 00fff6', ' 00fffc', ' 00fcff', ' 00f6ff', ' 00f0ff', ' 00eaff', ' 00e5ff', ' 00dfff', ' 00d9ff', ' 00d3ff', ' 00cdff', ' 00c7ff', ' 00c1ff', ' 00bbff', ' 00b5ff', ' 00afff', ' 00aaff', ' 00a4ff', ' 009eff', ' 0098ff', ' 0092ff', ' 008cff', ' 0086ff', ' 0080ff', ' 007aff', ' 0074ff', ' 006eff', ' 0069ff', ' 0063ff', ' 005dff', ' 0057ff', ' 0051ff', ' 004bff', ' 0045ff', ' 003fff', ' 0039ff', ' 0033ff', ' 002dff', ' 0028ff', ' 0022ff', ' 001cff', ' 0016ff', ' 0010ff', ' 020cff', ' 0408ff', ' 0604ff', ' 0800ff', ' 0e00ff', ' 1300ff', ' 1900ff', ' 1f00ff', ' 2500ff', ' 2b00ff', ' 3100ff', ' 3700ff', ' 3d00ff', ' 4300ff', ' 4900ff', ' 4f00ff', ' 5400ff', ' 5a00ff', ' 6000ff', ' 6600ff', ' 6c00ff', ' 7200ff', ' 7800ff', ' 7e00ff', ' 8400ff', ' 8a00ff', ' 9000ff', ' 9500ff', ' 9b00ff', ' a100ff', ' a700ff', ' ad00ff', ' b300ff', ' b900ff', ' bf00ff', ' c500ff', ' cb00ff', ' d000ff', ' d600ff', ' dc00ff', ' e200ff', ' e800ff', ' ee00ff', ' f400ff', ' f800fd', ' fa00f9', ' fc00f5', ' fe00f1', ' ff00ed', ' ff00e7', ' ff00e1', ' ff00db', ' ff00d5', ' ff00cf', ' ff00c9', ' ff00c3', ' ff00bd', ' ff00b7', ' ff00b1', ' ff00ac', ' ff00a6', ' ff00a0', ' ff009a', ' ff0094', ' ff008e', ' ff0088', ' ff0082', ' ff007c', ' ff0076', ' ff0071', ' ff006b', ' ff0065', ' ff005f', ' ff0059', ' ff0053', ' ff004d', ' ff0047', ' ff0041', ' ff003b', ' ff0035', ' ff0030', ' ff002a', ' ff0024', ' ff001e', ' ff0018'], 'min': 0.0, 'max': 3000.0, 'opacity': 1.0}

            #判断是哪个传感器，用不同的反演公式
            if id[11] == '5':
                # ssc的原始数据
                ssc_intial = l5_ssc(ocean)
                ssc = l5_ssc(ocean).visualize(**vis_params)

            elif id[11] == '7':
                # ssc的原始数据
                ssc_intial = l7_ssc(ocean)
                ssc = l7_ssc(ocean).visualize(**vis_params)

            elif id[11] == '8':
                # ssc的原始数据
                ssc_intial = l8_ssc(ocean)
                ssc = l8_ssc(ocean).visualize(**vis_params)

            Map.addLayer(ssc,{},'ssc')
            Map.addLayer(ssc_intial,{},'ssc_intial')
            
            # 将陆地设置为0，方便显示
            land = land.where(land,0)
            land = land.visualize(**vis_params0)
            Map.addLayer(land,{},'land')
            
            #为了便于成图陆地海洋合并成一张图             
            global blend 
            blend = land.blend(ssc)
            Map.addLayer(blend,{},'blend')
            
        except Exception as e:
            print(e)
            print('加载失败')
            
# 制图
def plt_show(d):
    ####警告信息框
    win32api.MessageBox(0, "制图功能仅在本地化软件中可用！", "提醒",win32con.MB_ICONWARNING)
    

# 回归评估指标计算，平均绝对误差，均方误差，均方根误差，平均相对误差
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=sum(absError)/len(absError)
    return mae,mse,RMSE

def compute_rsquared(X, Y):
    xBar = np.mean(X)
    yBar = np.mean(Y)
    SSR = 0
    varX = 0
    varY = 0
    for i in range(0, len(X)):
        diffXXBar = X[i] - xBar
        diffYYBar = Y[i] - yBar
        SSR += (diffXXBar * diffYYBar)
        varX += diffXXBar ** 2
        varY += diffYYBar ** 2

    SST = math.sqrt(varX * varY)
    r2=round((SSR / SST) ** 2,3)
    return r2

# 平均相对误差
def mean_relative_error(y_true, y_pred):
    relative_error = np.average(np.abs(y_true - y_pred) / y_true, axis=0)
    return relative_error

# 导入实测数据
def data_import(a1):
    global out_csv
    # 将实测数据对应点的预测值提取出来csv
    '''打开选择文件夹对话框'''
    root = tk.Tk()
    root.withdraw()

    # Folderpath = filedialog.askdirectory() #获得选择好的文件夹
    Filepath = filedialog.askopenfilename(title='选择实测数据的点shp文件', filetypes=[('SHP', '*.shp'), ('All Files', '*')]) #获得选择好的文件
    in_shp = Filepath
    # work_dir = os.path.expanduser('~/Downloads')
    # in_shp = os.path.join(work_dir, 'us_cities.shp')
    # if not os.path.exists(in_shp):
    #     data_url = 'https://github.com/giswqs/data/raw/main/us/us_cities.zip'
    #     geemap.download_from_url(data_url, out_dir=work_dir )
    shp_path.value = Filepath
    in_fc = geemap.shp_to_ee(in_shp)
    Map.addLayer(in_fc, {}, 'meature')

    #  Folderpath = filedialog.askdirectory() #获得选择好的文件夹
    # 默认将csv存在默认文件夹下
    Folderpath = os.path.expanduser('~/Downloads')
    work_dir = Folderpath
    out_csv = os.path.join(work_dir, 'point.csv')
    geemap.extract_values_to_points(in_fc, ssc_intial, out_csv)

# 精度验证
def yanzheng(a2):
    data = pd.read_csv(out_csv)
    col_1 = data["z"]  #获取一列，用一维数据
    testX = np.array(col_1)
    col_12 = data["constant"]  #获取di两列
    testY = np.array(col_12)
    
    mae,mse,rmse=compute_mae_mse_rmse(testX,testY)
    r2=compute_rsquared(testX,testY)
    mre = mean_relative_error(testX,testY)
    
    yz_result = "R²(决定系数)：%.4f " %r2  + '\n' + "MAE(平均绝对误差):%.4f " %mae + '\n'  + "MRE(平均相对误差):%.4f "%mre + '\n' + "MSE(均方误差):%.4f " %mse + '\n'  + "RMSE(均方根误差):%.4f "%rmse 
    
    yanzheng_result.value = yz_result

# 下载导出遥感影像
def eximg(a3):
    # roi直接用黄河口边界
    roi = ee.Geometry.Polygon(
        [[[118.0589, 38.4364],
          [120.2327, 38.1181],
            [119.7274, 36.5238],
            [117.5964, 36.8278],
            [118.0589, 38.4364]]], None, False)
    
    '''打开选择文件夹对话框'''
    root = tk.Tk()
    root.withdraw()

    # Folderpath = filedialog.askdirectory() #获得选择好的文件夹
    Filepath = filedialog.asksaveasfilename(title='保存遥感影像', filetypes=[('TIF', '*.tif'), ('All Files', '*')]) #获得选择好的文件
    filename = Filepath + '.tif'

    geemap.ee_export_image(image, filename=filename, scale=110, region=roi, file_per_band=False)
    
# 下载导出反演影像
def exssc(a3):
    # roi直接用黄河口边界
    roi = ee.Geometry.Polygon(
        [[[118.0589, 38.4364],
          [120.2327, 38.1181],
            [119.7274, 36.5238],
            [117.5964, 36.8278],
            [118.0589, 38.4364]]], None, False)
    
    '''打开选择文件夹对话框'''
    root = tk.Tk()
    root.withdraw()

    # Folderpath = filedialog.askdirectory() #获得选择好的文件夹
    Filepath = filedialog.asksaveasfilename(title='保存反演影像', filetypes=[('TIF', '*.tif'), ('All Files', '*')]) #获得选择好的文件
    filename = Filepath + '.tif'

    geemap.ee_export_image(ssc_intial, filename=filename, scale=110, region=roi, file_per_band=False)
# 地图初始化
def map_zero(e):
    time_slider.value = 1984
    cloud_slider.value = 10
    result_ta.value = ""
    image_cb.value = ""
    yanzheng_result.value =""
    image_cb.options = []
    Map.clear_layers()
    Map.add_basemap('Google Maps')
    out.clear_output()
#     bj.close()
#     tt = ui()
#     tt
    
    
# 选择保存路径
def select_path(f):
    '''打开选择文件夹对话框'''
    root = tk.Tk()
    root.withdraw()

    Folderpath = filedialog.askdirectory() #获得选择好的文件夹
    # Filepath = filedialog.askopenfilename() #获得选择好的文件
    work_dir = Folderpath
    path.value = work_dir

#影像GIF
def imagegif(g):
    fontcolor = color.value
    title0 = title.value
    speed0 = speed.value
    work_dir = path.value
    start_year = time_range.value[0]
    end_year = time_range.value[1]
    # 制作影像动态图
    images = []  #用来存储所有的影像id
    # 用影像Id将所有影像合成一个collection
    for i in range(start_year,end_year):
        sys_id = cloud_select_first(i)
        image = ee.Image(sys_id)
        # 判断是不是L7影像，进行去条带操作
        if sys_id[11] == '7':
            image = func_lvr(image)
        images.append(image)
    collection = ee.ImageCollection(images)

    # roi直接用黄河口边界
    roi = ee.Geometry.Polygon(
        [[[118.0589, 38.4364],
          [120.2327, 38.1181],
            [119.7274, 36.5238],
            [117.5964, 36.8278],
            [118.0589, 38.4364]]], None, False)

    # 定义动图参数
    video_args = {
        'dimensions': 600,
        'region': roi,
        'framePerSecond': speed0,
        'bands': ['B4','B3','B2'],
        'min': 0,
        'max': 4000,
        'gamma': [1,1,1]
    }

    vv = collection.getVideoThumbURL(video_args)
    geemap.download_from_url(vv, out_dir=work_dir, unzip = False, out_file_name = "test00.gif")

    out_gif = os.path.join(work_dir, "test00.gif")

    geemap.add_text_to_gif(out_gif, out_gif, xy=('3%', '5%'), text_sequence=start_year, font_size=30, font_color=fontcolor, add_progress_bar=False)

    label = title0
    geemap.add_text_to_gif(out_gif, out_gif, xy=('2%', '88%'), text_sequence=label, font_size=30, font_color=fontcolor, progress_bar_color='cyan')

#     geemap.show_image(out_gif)
    with out:
        geemap.show_image(out_gif)
    out

#SSC-GIF
def sscgif(h):
    fontcolor = color.value
    title0 = title.value
    speed0 = speed.value
    work_dir = path.value
    start_year = time_range.value[0]
    end_year = time_range.value[1]
    # 制作SSC动态图
    SSCs = []  #用来存储所有的影像id
    # 用影像Id将所有影像合成一个collection
    for i in range(start_year,end_year):
        sys_id = cloud_select_first(i)
        image = ee.Image(sys_id)
        # 判断是不是L7影像，进行去条带操作
        if sys_id[11] == '7':
            image = func_lvr(image)

        landshp = ee.FeatureCollection('users/bluesky666/yrd_1984')
        oceanshp = ee.FeatureCollection('users/bluesky666/yrd_1984sea')
        # 利用陆地和海洋shp分别裁剪出陆地和海洋范围
        land = image.clip(landshp)
        ocean = image.clip(oceanshp)

        # Set visualization parameters.
        vis_params0 = {'bands': ['B1'], 'min': 0.0, 'max': 100.0, 'opacity': 1.0, 'gamma': 1.0}
        vis_params = {'bands': ['constant'], 'palette': ['ff0000', ' ff0600', ' ff0c00', ' ff1200', ' ff1800', ' ff1e00', ' ff2300', ' ff2900', ' ff2f00', ' ff3500', ' ff3b00', ' ff4100', ' ff4700', ' ff4d00', ' ff5300', ' ff5900', ' ff5f00', ' ff6400', ' ff6a00', ' ff7000', ' ff7600', ' ff7c00', ' ff8200', ' ff8800', ' ff8e00', ' ff9400', ' ff9a00', ' ff9f00', ' ffa500', ' ffab00', ' ffb100', ' ffb700', ' ffbd00', ' ffc300', ' ffc900', ' ffcf00', ' ffd500', ' ffdb00', ' ffe000', ' ffe600', ' ffec00', ' fef100', ' fcf500', ' faf900', ' f8fd00', ' f4ff00', ' eeff00', ' e8ff00', ' e2ff00', ' ddff00', ' d7ff00', ' d1ff00', ' cbff00', ' c5ff00', ' bfff00', ' b9ff00', ' b3ff00', ' adff00', ' a7ff00', ' a2ff00', ' 9cff00', ' 96ff00', ' 90ff00', ' 8aff00', ' 84ff00', ' 7eff00', ' 78ff00', ' 72ff00', ' 6cff00', ' 66ff00', ' 61ff00', ' 5bff00', ' 55ff00', ' 4fff00', ' 49ff00', ' 43ff00', ' 3dff00', ' 37ff00', ' 31ff00', ' 2bff00', ' 25ff00', ' 20ff00', ' 1aff00', ' 14ff00', ' 0eff00', ' 08ff00', ' 06ff04', ' 04ff08', ' 02ff0c', ' 00ff10', ' 00ff16', ' 00ff1b', ' 00ff21', ' 00ff27', ' 00ff2d', ' 00ff33', ' 00ff39', ' 00ff3f', ' 00ff45', ' 00ff4b', ' 00ff51', ' 00ff57', ' 00ff5c', ' 00ff62', ' 00ff68', ' 00ff6e', ' 00ff74', ' 00ff7a', ' 00ff80', ' 00ff86', ' 00ff8c', ' 00ff92', ' 00ff97', ' 00ff9d', ' 00ffa3', ' 00ffa9', ' 00ffaf', ' 00ffb5', ' 00ffbb', ' 00ffc1', ' 00ffc7', ' 00ffcd', ' 00ffd3', ' 00ffd8', ' 00ffde', ' 00ffe4', ' 00ffea', ' 00fff0', ' 00fff6', ' 00fffc', ' 00fcff', ' 00f6ff', ' 00f0ff', ' 00eaff', ' 00e5ff', ' 00dfff', ' 00d9ff', ' 00d3ff', ' 00cdff', ' 00c7ff', ' 00c1ff', ' 00bbff', ' 00b5ff', ' 00afff', ' 00aaff', ' 00a4ff', ' 009eff', ' 0098ff', ' 0092ff', ' 008cff', ' 0086ff', ' 0080ff', ' 007aff', ' 0074ff', ' 006eff', ' 0069ff', ' 0063ff', ' 005dff', ' 0057ff', ' 0051ff', ' 004bff', ' 0045ff', ' 003fff', ' 0039ff', ' 0033ff', ' 002dff', ' 0028ff', ' 0022ff', ' 001cff', ' 0016ff', ' 0010ff', ' 020cff', ' 0408ff', ' 0604ff', ' 0800ff', ' 0e00ff', ' 1300ff', ' 1900ff', ' 1f00ff', ' 2500ff', ' 2b00ff', ' 3100ff', ' 3700ff', ' 3d00ff', ' 4300ff', ' 4900ff', ' 4f00ff', ' 5400ff', ' 5a00ff', ' 6000ff', ' 6600ff', ' 6c00ff', ' 7200ff', ' 7800ff', ' 7e00ff', ' 8400ff', ' 8a00ff', ' 9000ff', ' 9500ff', ' 9b00ff', ' a100ff', ' a700ff', ' ad00ff', ' b300ff', ' b900ff', ' bf00ff', ' c500ff', ' cb00ff', ' d000ff', ' d600ff', ' dc00ff', ' e200ff', ' e800ff', ' ee00ff', ' f400ff', ' f800fd', ' fa00f9', ' fc00f5', ' fe00f1', ' ff00ed', ' ff00e7', ' ff00e1', ' ff00db', ' ff00d5', ' ff00cf', ' ff00c9', ' ff00c3', ' ff00bd', ' ff00b7', ' ff00b1', ' ff00ac', ' ff00a6', ' ff00a0', ' ff009a', ' ff0094', ' ff008e', ' ff0088', ' ff0082', ' ff007c', ' ff0076', ' ff0071', ' ff006b', ' ff0065', ' ff005f', ' ff0059', ' ff0053', ' ff004d', ' ff0047', ' ff0041', ' ff003b', ' ff0035', ' ff0030', ' ff002a', ' ff0024', ' ff001e', ' ff0018'], 'min': 0.0, 'max': 3000.0, 'opacity': 1.0}


        #判断是哪个传感器，用不同的反演公式
        if sys_id[11] == '5':
            ssc = l5_ssc(ocean).visualize(**vis_params)

        elif sys_id[11] == '7':
            ssc = l7_ssc(ocean).visualize(**vis_params)

        elif sys_id[11] == '8':
            ssc = l8_ssc(ocean).visualize(**vis_params)

    #     Map.addLayer(ssc,{},'ssc')

        # 将陆地设置为0，方便显示
        land = land.where(land,0)
        land = land.visualize(**vis_params0)
    #     Map.addLayer(land,{},'land')

        #为了便于成图陆地海洋合并成一张图             
        global blend 
        blend = land.blend(ssc)
    #     Map.addLayer(blend,{},'blend')


        SSCs.append(blend)

    collection = ee.ImageCollection(SSCs)

    # roi直接用黄河口边界
    roi = ee.Geometry.Polygon(
        [[[118.0589, 38.4364],
          [120.2327, 38.1181],
            [119.7274, 36.5238],
            [117.5964, 36.8278],
            [118.0589, 38.4364]]], None, False)

    # 定义动图参数
    video_args = {
        'dimensions': 600,
        'region': roi,
        'framePerSecond': speed0,
        'bands': ['vis-red','vis-green','vis-blue'],
        'min': 0,
        'max': 100,
        'gamma': [1,1,1]
    }

    vv = collection.getVideoThumbURL(video_args)
    geemap.download_from_url(vv, out_dir=work_dir, unzip = False, out_file_name = "test001.gif")

    out_gif = os.path.join(work_dir, "test001.gif")

    geemap.add_text_to_gif(out_gif, out_gif, xy=('3%', '5%'), text_sequence=start_year, font_size=30, font_color=fontcolor, add_progress_bar=False)

    label = title0
    geemap.add_text_to_gif(out_gif, out_gif, xy=('2%', '88%'), text_sequence=label, font_size=30, font_color=fontcolor, progress_bar_color='cyan')
    with out:
        geemap.show_image(out_gif)
    out
    
# 增加点击事件
static_btn.on_click(submit_clicked)
image_btn.on_click(show_image)
ssc_btn.on_click(show_ssc)
plot_btn.on_click(plt_show)
sc_btn.on_click(data_import)
yanzheng_btn.on_click(yanzheng)
exportimg_btn.on_click(eximg)
exportssc_btn.on_click(exssc)
zero_btn.on_click(map_zero)
seletpath_btn.on_click(select_path)
imagegif_btn.on_click(imagegif)
sscgif_btn.on_click(sscgif)